In [21]:
!nvidia-smi

Sat Apr  9 06:49:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    73W / 149W |   5602MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
! pip install transformers==4.0.1

In [24]:
! pip install torch==1.4.0

In [25]:
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

config = {
    'train_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/train.csv',
    'test_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/test.csv',
    'train_val_ratio':0.1,
    'model_path':'/content/drive/MyDrive/Colab Notebooks/dataset/NeZha_model',
    'batch_size':16,
    'head': 'CNN',
    'num_epochs':1,
    'warmup_ratio':0.1, # warm up
    'learning_rate':2e-5,
    'logging_step':500,
    'seed':2022
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  return seed

seed_everything(config['seed'])

2022

In [26]:
from collections import defaultdict
def read_data(config, tokenizer, mode = 'train'):
  data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
  if mode == 'train':
    X_train, y_train = defaultdict(list),[]
    X_val, y_val = defaultdict(list),[]
    num_val = int(len(data_df) * config['train_val_ratio'])
  else:
    X_test, y_test = defaultdict(list),[]

  for i, row in tqdm(data_df.iterrows(), desc=f'preprocess {mode} data', colour = 'blue', total = len(data_df)):
    label = row[1] if mode == 'train' else 0
    sentence = row[-1]

    inputs = tokenizer.encode_plus(sentence, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

    if mode == 'train':
      if i < num_val:
        X_val['inputs_ids'].append(inputs['input_ids'])
        y_val.append(label)
        X_val['token_type_ids'].append(inputs['token_type_ids'])
        X_val['attention_mask'].append(inputs['attention_mask'])
      else:
        X_train['inputs_ids'].append(inputs['input_ids'])
        y_train.append(label)
        X_train['token_type_ids'].append(inputs['token_type_ids'])
        X_train['attention_mask'].append(inputs['attention_mask'])

    else:
        X_test['inputs_ids'].append(inputs['input_ids'])
        y_test.append(label)
        X_test['token_type_ids'].append(inputs['token_type_ids'])
        X_test['attention_mask'].append(inputs['attention_mask'])

  if mode == 'train':
    label2id = {label: i for i, label in enumerate(np.unique(y_train))}
    id2label = {i: label for label, i in label2id.items()}

    y_train = torch.tensor([label2id[i] for i in y_train], dtype =torch.long)

    y_val = torch.tensor([label2id[i] for i in y_val], dtype =torch.long)
    return X_train, y_train, X_val, y_val, label2id, id2label

  else:
    y_test = torch.tensor(y_test, dtype = torch.long)
    return X_test, y_test


In [27]:
from torch.utils.data import Dataset
class TNEWSData(Dataset):
  def __init__(self, X, y):
    self.x = X
    self.y = y
  
 
  def __getitem__(self, idx):
    return{
        'inputs_ids': self.x['inputs_ids'][idx],
        'label':self.y[idx],
        'token_type_ids':self.x['token_type_ids'][idx],
        'attention_mask':self.x['attention_mask'][idx]

    }

 
  def __len__(self):
    return self.y.size(0)

In [28]:
# 从 TNEWSDataset 返回的多个example，合并为Tensor
def collate_fn(example):
  input_ids_list = []
  labels = []
  token_type_ids_list = []
  attention_mask_list = []

  for ex in example:
    input_ids_list.append(ex['inputs_ids'])
    labels.append(ex['label'])
    token_type_ids_list.append(ex['token_type_ids'])
    attention_mask_list.append(ex['attention_mask'])

  max_len = max(len(input_ids) for input_ids in input_ids_list)
  input_ids_tensor = torch.zeros((len(labels), max_len),dtype=torch.long)
  token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
  attention_mask_tensor = torch.zeros_like(input_ids_tensor)

  for i, input_ids in enumerate(input_ids_list):
    input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype = torch.long)
    token_type_ids_tensor[i, :len(input_ids)] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
    attention_mask_tensor[i, :len(input_ids)] = torch.tensor(attention_mask_list[i], dtype = torch.long)

  return {
      'input_ids': input_ids_tensor,
      'labels': torch.tensor(labels ,dtype= torch.long),
      'token_type_ids':token_type_ids_tensor,
      'attention_mask':attention_mask_tensor
  }  

In [29]:
# dataloader 可以并行
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
  tokenizer = BertTokenizer.from_pretrained(config['model_path'])
  X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
  X_test, y_test = read_data(config, tokenizer, mode='test')

  train_dataset = TNEWSData(X_train, y_train)
  val_dataset = TNEWSData(X_val, y_val)
  test_dataset = TNEWSData(X_test, y_test)

  train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = True, collate_fn=collate_fn)
  val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = False, collate_fn=collate_fn)
  test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = False, collate_fn=collate_fn)

  return train_dataloader, val_dataloader, test_dataloader, id2label


In [30]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

preprocess test data: 100%|██████████| 10000/10000 [00:05<00:00, 1784.76it/s]


In [31]:
import torch.nn.functional as F
import torch.nn as nn
from NeZha import *
class NeZhaForTNEWS(NeZhaPreTrainedModel):
  def __init__(self, config, model_path, classifier):
    super(NeZhaForTNEWS, self).__init__(config)

    self.nezha = NeZhaModel.from_pretrained(model_path, config=config)
    self.classifier = classifier  # head
    self.config = config

  def forward(self, input_ids, token_type_ids, attention_mask, labels):
    outputs = self.nezha(input_ids = input_ids,
                token_type_ids = token_type_ids,
                attention_mask = attention_mask)
    hidden_states = outputs[2]

    logits = self.classifier(hidden_states, input_ids)

    outputs = (logits, )

    if labels is not None:
      loss_fct = FocalLoss(num_classes=self.config.num_labels)
      loss = loss_fct(logits, labels.view(-1))
      outputs = (loss, )+ outputs

    return outputs

## typing模块的作用
- 类型检查，防止运行时出现参数和返回值类型不符合。
- 作为开发文档附加说明，方便使用者调用时传入和返回参数类型。
- 该模块加入后并不会影响程序的运行，不会报正式的错误，只有提醒。

### 说明：

- 在传入参数时通过“参数名:类型”的形式声明参数的类型；
- 返回结果通过"-> 结果类型"的形式声明结果的类型。
- 在调用的时候如果参数的类型不正确pycharm会有提醒，但不会影响程序的运行。
- 对于如list列表等，还可以规定得更加具体一些，如：“-> List[str]”,规定返回的是列表，并且元素是字符串。

*举例*
```
from typing import List, Tuple, Dict
def add(a:int, string:str, f:float, b:bool) -> Tuple[List, Tuple, Dict, bool]:
list1 = list(range(a))
    tup = (string, string, string)
    d = {"a":f}
    bl = b
    return list1, tup, d,bl
print(add(5,"hhhh", 2.3, False))
```
结果：([0, 1, 2, 3, 4], ('hhhh', 'hhhh', 'hhhh'), {'a': 2.3}, False)

In [32]:
from typing import List
class ConvClassifier(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.conv = nn.Conv1d(in_channels = config.hidden_size, out_channels = config.hidden_size, kernel_size = 3, padding=(3 - 1) // 2)
    self.global_max_pool = nn.AdaptiveMaxPool1d(1)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.fc = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self, hidden_states: List[torch.Tensor], input_ids: torch.Tensor):
    hidden_states = self.dropout(hidden_states[-1])
    hidden_states = hidden_states.permute(0 ,2, 1)

    out = F.relu(self.conv(hidden_states))
    out = self.global_max_pool(out).squeeze(dim=2)
    out = self.fc(out)
    
    return out

In [33]:
def build_model(model_path, config, head):
  heads = {
      'CNN': ConvClassifier
  }
  assert head in heads ,"head must have been implemented"
  print(f'>>> You are using {head} head, please wait...')
  model = NeZhaForTNEWS(config, model_path, heads[head](config))
  return model

In [34]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
  model.eval()
  preds = []
  labels = []
  val_loss = 0.
  val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

  with torch.no_grad():
    for batch in val_iterator:
      labels.append(batch['labels'])
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      loss, logits = model(**batch)[:2]
      val_loss += loss.item()
      preds.append(logits.argmax(dim = -1).detach().cpu())

  avg_val_loss = val_loss / len(val_dataloader)
  labels = torch.cat(labels, dim = 0).numpy()
  preds = torch.cat(preds, dim = 0).numpy()
  f1 = f1_score(labels, preds, average='macro')
  return avg_val_loss, f1


In [35]:
from extra_loss import *
from extra_optim import *
from transformers import AdamW
from tqdm import trange
def train(config, id2label, train_dataloader, val_dataloader):
  nezha_config = NeZhaConfig.from_pretrained(config['model_path'])
  nezha_config.output_hidden_states = True
  nezha_config.num_labels = len(id2label)

  model = build_model(config['model_path'], nezha_config, config['head'])

  # 得到模型的参数
  optimizer_grouped_parameters = model.parameters()
  # 定义一个基优化器
  optimizer = AdamW(model.parameters(), lr= config['learning_rate'])
  # Lookahead要有一个基优化器， k=5, alpha=1
  optimizer = Lookahead(optimizer, 5, 1)
  total_steps = config['num_epochs'] * len(train_dataloader)
  # 每调用warmup_steps次， 对应的学习率就会调整一次
  lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps = int(config['learning_rate'] * total_steps), t_total = total_steps)
                                                                   

  model.to(config['device'])
  epoch_iterator = trange(config['num_epochs'])
  global_steps = 0
  train_loss = 0.
  logging_loss = 0.

  for epoch in epoch_iterator:
    train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
    model.train()
    for batch in train_iterator:
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      loss = model(**batch)[0]
      model.zero_grad()
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      global_steps += 1

      if global_steps % config['logging_step'] == 0:
        print_train_loss = (train_loss - logging_loss) / config['logging_step']
        logging_loss = train_loss
        avg_val_loss, f1 = evaluation(config, model, val_dataloader)

        print_log = f'>>>traing loss:{print_train_loss: .5f}, valid loss:{avg_val_loss: .5f}, valid f1 score:{f1: .5f}'
        print(print_log)
        model.train()

  return model    

In [36]:
model = train(config, id2label, train_dataloader, val_dataloader)

>>> You are using CNN head, please wait...


Some weights of NeZhaModel were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/NeZha_model and are newly initialized: ['bert.encoder.layer.0.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.1.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.2.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.3.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.4.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.5.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.6.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.7.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.8.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.9.attention.self.relative_positions_encodi

>>>traing loss: 0.96119, valid loss: 0.72347, valid f1 score: 0.47554



Training:  33%|███▎      | 999/3002 [05:25<09:37,  3.47it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:51,  2.98it/s]

Evaluation:   1%|          | 2/334 [00:00<01:09,  4.74it/s]

Evaluation:   1%|          | 4/334 [00:00<00:42,  7.68it/s]

Evaluation:   2%|▏         | 6/334 [00:00<00:34,  9.52it/s]

Evaluation:   2%|▏         | 8/334 [00:00<00:30, 10.82it/s]

Evaluation:   3%|▎         | 10/334 [00:01<00:29, 11.15it/s]

Evaluation:   4%|▎         | 12/334 [00:01<00:27, 11.54it/s]

Evaluation:   4%|▍         | 14/334 [00:01<00:27, 11.60it/s]

Evaluation:   5%|▍         | 16/334 [00:01<00:26, 12.10it/s]

Evaluation:   5%|▌         | 18/334 [00:01<00:26, 11.89it/s]

Evaluation:   6%|▌         | 20/334 [00:01<00:26, 12.00it/s]

Evaluation:   7%|▋         | 22/334 [00:02<00:25, 12.03it/s]

Evaluation:   7%|▋         | 24/334 [00:02<00:26, 11.57it/s]

Evaluation:   8%|▊         | 26/334 [00:02<00:27, 11.27it/s]

Evaluation:   8%|▊   

>>>traing loss: 0.72028, valid loss: 0.66758, valid f1 score: 0.49586


...


Training:  83%|████████▎ | 2499/3002 [14:17<02:25,  3.46it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:50,  3.01it/s]

Evaluation:   1%|          | 2/334 [00:00<01:06,  5.03it/s]

Evaluation:   1%|          | 4/334 [00:00<00:43,  7.57it/s]

Evaluation:   2%|▏         | 6/334 [00:00<00:34,  9.42it/s]

Evaluation:   2%|▏         | 8/334 [00:00<00:30, 10.79it/s]

Evaluation:   3%|▎         | 10/334 [00:01<00:28, 11.23it/s]

Evaluation:   4%|▎         | 12/334 [00:01<00:28, 11.45it/s]

Evaluation:   4%|▍         | 14/334 [00:01<00:27, 11.57it/s]

Evaluation:   5%|▍         | 16/334 [00:01<00:27, 11.75it/s]

Evaluation:   5%|▌         | 18/334 [00:01<00:26, 11.76it/s]

Evaluation:   6%|▌         | 20/334 [00:01<00:26, 11.95it/s]

Evaluation:   7%|▋         | 22/334 [00:02<00:26, 11.85it/s]

Evaluation:   7%|▋         | 24/334 [00:02<00:26, 11.67it/s]

Evaluation:   8%|▊         | 26/334 [00:02<00:27, 11.32it/s]

Evaluation:   8%|▊  

>>>traing loss: 0.61215, valid loss: 0.60698, valid f1 score: 0.53335



Training: 100%|█████████▉| 2999/3002 [17:13<00:00,  3.68it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:49,  3.05it/s]

Evaluation:   1%|          | 2/334 [00:00<01:06,  4.99it/s]

Evaluation:   1%|          | 3/334 [00:00<00:51,  6.44it/s]

Evaluation:   1%|▏         | 5/334 [00:00<00:37,  8.79it/s]

Evaluation:   2%|▏         | 7/334 [00:00<00:31, 10.41it/s]

Evaluation:   3%|▎         | 9/334 [00:01<00:28, 11.40it/s]

Evaluation:   3%|▎         | 11/334 [00:01<00:28, 11.29it/s]

Evaluation:   4%|▍         | 13/334 [00:01<00:28, 11.37it/s]

Evaluation:   4%|▍         | 15/334 [00:01<00:26, 11.92it/s]

Evaluation:   5%|▌         | 17/334 [00:01<00:26, 11.88it/s]

Evaluation:   6%|▌         | 19/334 [00:01<00:26, 11.97it/s]

Evaluation:   6%|▋         | 21/334 [00:02<00:26, 11.98it/s]

Evaluation:   7%|▋         | 23/334 [00:02<00:26, 11.70it/s]

Evaluation:   7%|▋         | 25/334 [00:02<00:26, 11.78it/s]

Evaluation:   8%|▊   

>>>traing loss: 0.58535, valid loss: 0.60679, valid f1 score: 0.52051



100%|██████████| 1/1 [17:45<00:00, 1065.07s/it]


In [37]:
def prediction(config, id2label, model, test_dataloader):
  test_iterator = tqdm(test_dataloader, desc='Prediction', total = len(test_dataloader))
  model.eval()
  test_preds = []

  with torch.no_grad():
    for batch in test_iterator:
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      logits = model(**batch)[1]
      test_preds.append(logits.argmax(dim=-1).detach().cpu())
  
  test_preds = torch.cat(test_preds, dim=0).numpy()
  test_preds = [id2label[id_] for id_ in test_preds]

  test_df = pd.read_csv(config['test_file_path'], sep=',')
  test_df.insert(1, column='label', value=test_preds)
  test_df.drop(['sentence'], 1,inplace=True)#1表示按列删除
  test_df.to_csv('submission_Nezha_FocalLoss.csv', index=False, encoding= 'utf8')

In [38]:
prediction(config, id2label, model, test_dataloader)

Prediction: 100%|██████████| 625/625 [00:53<00:00, 11.76it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
